In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

# Linear Regression

Linear regression is a method to model the relation between values:


*   Input data $A = \begin{bmatrix}
a_1 & a_2 & \cdots & a_n
\end{bmatrix}$ where columns $a_i$'s are different data features;
*   Output data $y$ which is a column vector.

The linear regression model assumes that the output $y$ is a linear function in terms of the input variables. So basically it can be formulated into a linear system

$$y = Ax$$

where the vector $x$ is a vector of coefficients (or weights of this linear model) which reveals how output $y$ relates *linearly* with the input $A$, and it is the vector we would want to solve.

You may want to say, well this is a linear system, why not just solve it? The fact is that, in general, there are more equations than the number of unknowns (data features, number of columns of input $A$), which means that the linear system we have is often "overdetermined", and inconsistent.

The way to achieve the "best approximation" of the solution is when the values for $x$ in the linear model minimize the square error: find $x$ such that the norm

$$||y - Ax||^2 = (y - Ax) \cdot (y - Ax).$$

We will see in the Section of least square problems that the least square solution $x$ to a generally inconsistent linear model $Ax = y$ is unique when columns of $A$ are linearly independent. The key point to get such best approximation $x$ is to modify the linear system as the following *consistent* linear system:

$$\widehat{y} = \text{proj}_{\text{Col}(A)} y = Ax$$

and solve it. It is the same as solving the normal equation:

$$(A^T A)x = A^T y \implies x = (A^T A)^{-1}A^T y.$$

The presense of the inverse $(A^T A)^{-1}$ is potentially the challanging part.



## Example

Let consider a simple linear regression problem where there is a single feature, namely, $A$ and $y$ are both column vectors. We aim to find a scalar $x \in \mathbb{R}$ which describes the linear relation between $A$ and $y$ the best.

In [ ]:
# define input and output data
data = np.array([
    [1.03, 3.46],
    [2.32, 5.78],
    [2.98, 9.98],
    [3.67, 10.56],
    [4.39, 14.09],
    [5.64, 15.21]
])
A, y = data[:, 0], data[:, 1]
A = A.reshape(len(A), 1)
plt.scatter(A, y)
plt.show()

## Inverse

The easiest approach is to solve its normal equation directly as mentioned above:

$$x = (A^TA)^{-1}A^Ty.$$

In [ ]:
# solve the normal equation directly
# least square solution
x = la.inv(A.T @ A) @ A.T @ y
print(x)
# use x to predict the output
yhat = A @ x
# plot the graph with the predicted line
plt.scatter(A, y)
plt.plot(A, yhat, 'r')
plt.show()

## QR decomposition

Another approach is using QR decomposition of $A$ when columns of $A$ are linearly independent.

Recall how we obtain the QR factorization for a tall matrix $A$ whose columns are linearly independent. First we apply the Gram-Schmidt process to the column vectors of $A$ to get an orthonormal basis for the column space $\text{Col}(A)$. We denote by $Q$ the matrix whose columns are formed by the orthonormal basis elements. Then the QR factorization of $A$ is

$$A = QR$$

where $R$ is a upper triangular matrix, which is a square matrix, invertible, with strictly positive main diagonal entries.

Using the decomposition, to solve the least square problem $y = Ax$ we obtain

$$Q^T y = Q^TQR x \implies x = R^{-1}Q^T y$$

since $Q$ is an orthogonal matrix, $Q^TQ = I$, and $R$ is invertible. Note that the left-hand-side $Q^T y = \text{proj}_{\text{Col}(A)} y$.

In [ ]:
# use QR decomposition
Q, R = la.qr(A)
x_QR = la.inv(R) @ Q.T @ y
print(x_QR)
# use x_QR to predict the output
yhat = A @ x_QR
# plot the graph with the predicted line
plt.scatter(A, y)
plt.plot(A, yhat, 'r')
plt.show()

## SVD and Pseudo-inverse

Let $A$ be an $n \times m$ real matrix. The SVD (singular value decomposition) of $A$ is

$$A = U \Sigma V^T$$

where

*   $U$ is a square matrix of size $n \times n$;
*   $\Sigma$ is an $n \times m$ matrix which is diagonal;
*   $V^T$ is another square matrix of size $m \times m$;
*   Both $U$ and $V$ are orthogonal matrices. The diagonal entries of $\Sigma$ are called *singular values* which are always nonnegative.

By viewing $A = U\Sigma V^T$ as a linear transformation and a composition of three linear transformations, it can be considered as rotation (represented by $V^T$), followed by stretching by singular values (represented by $\Sigma$), and then followed by another rotation (represented by $U$).

Note that $A$ has to have linearly independent column vectors in order to admit a QR factorization, but there is no requirement on $A$ to get a singular value decomposition. Thus SVD is more general and stable.

We will use SVD to find inverse of matrix $A^T A$ (if exists, with size $m \times m$) and take advantage of $U^T = U^{-1}$ if $U$ is a square orthogonal matrix. Assume SVD of $A^T A$ is

$$A^T A = U \Sigma V^T \implies (A^T A)^{-1} = V\Sigma^{-1} U^T,$$

where since $\Sigma$ is square diagonal with main diagonal entries positive,

$$\Sigma^{-1} = \begin{bmatrix}
1/s_1 & 0 & \cdots & 0 \\
0 & 1/s_2 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 1/s_m
\end{bmatrix}.$$

In general, not all singular values are strictly positive, which means that $\Sigma$ may not be invertible. This happens when the square matrix $A^TA$ is not invertible. The most straightforward way to remedy this situation is the following. Suppose

$$\Sigma = \begin{bmatrix}
s_1 & & & & & \\
 & \ddots & & & & \\
 & & s_k & & & \\
 & & & 0 & & \\
 & & & & \ddots & \\
 & & & & & 0
\end{bmatrix},$$

where $s_1,\cdots,s_k$ are strictly positive.
We define its *pseudo-inverse*, denoted by $\Sigma^\dagger$ as

$$\Sigma^\dagger = \begin{bmatrix}
1/s_1 & & & & & \\
 & \ddots & & & & \\
 & & 1/s_k & & & \\
 & & & 0 & & \\
 & & & & \ddots & \\
 & & & & & 0
\end{bmatrix},$$

Thus when $(A^TA)$ is not invertible, we can still take its pseudo-inverse, together with SVD, it is given by

$$(A^T A)^\dagger = V \Sigma^\dagger U^T,$$

and the solution to the normal equation is
$$x = (A^T A)^\dagger A^T y.$$

In [ ]:
# use singular value decomposition (SVD)
# the function la.pinv is a built-in computing pseudo-inverse of a matrix
x_SVD = la.pinv(A.T @ A) @ A.T @ y
print(x_SVD)
# use x_SVD to predict the output
yhat = A @ x_SVD
# plot the graph with the predicted line
plt.scatter(A, y)
plt.plot(A, yhat, 'r')
plt.show()

## Problem

Let $L_1, L_2, \cdots, L_7$ be seven lines in $\mathbb{R}^6$. Find a point $x \in \mathbb{R}^6$ which minimizes the sum of distances to all the seven lines $L_1, L_2,\cdots, L_7$.

### Setup

1.   We all know that two distinct points define a line. To describe a line $L$ in $\mathbb{R}^6$, let $v$ and $w$ be two distinct points on $L$. Then we have a direction vector of the line $L$ given by

$$r = \dfrac{v - w}{||v - w||}.$$

2.   There is matrix $(I - rr^T)$ which is called the projection matrix. For any vector $p \in \mathbb{R}^6$,

$$(I - rr^T)p = p - r(r^Tp) = p - (p \cdot r)r = p - \text{proj}_{r}p$$

which means that the matrix $(I - rr^T)$ projects each vector $p$ to the subspace $W$ which is orthogonal to $L$.

3.   Using the projection matrix $(I - rr^T)$, the intersection of $L$ with the subspace $W$ is given by the projection of $v$ (or $w$) onto $W$:

$$q = (I - rr^T)v.$$

4.   Thus for a point $x \in \mathbb{R}^6$, its distance to the line $L$ can be computed as distance of its projection to $W$ to the intersection point $q$. In other words, we express the distance between line $L$ and point $p$, as length of line segments lying inside $W$ with endpoints $q$ and $(I - rr^T)x$:

$$||q - (I - rr^T)x||.$$

Applying to a set of seven lines $L_1,\cdots,L_7$, we want to find point $x \in \mathbb{R}^6$ which minimizes

$$\sum_{i = 1}^7 ||q_i - (I - r_ir_i^T)x||^2$$

which can be written as a quadratic form in terms of the vector $x$:

$$\sum_{i = 1}^7 ||q_i - (I - r_ir_i^T)x||^2 = x^T A x - 2b^T x + c$$

where

*   $A = \sum_{i = 1}^7 (I - rr^T)$ is of size $7 \times 7$;
*   $b = \sum_{i = 1}^7 q_i$ is a vector;
*   $c = \sum_{i = 1}^7 q_i^T q_i$ is a scalar.

Since $x$ which minimizes $x^T A x - 2b^T x + c$ is precisely given by the least square solution to $Ax = b$,

**Use the above three methods (directly solving normal equation, QR decomposition, SVD) to find such minimizer $x$.**



In [ ]:
# setup
# generate seven lines in R^6
rng = np.random.default_rng(seed=3264)
L = rng.uniform(low=-10, high=10, size=(7, 2, 6))  # (lines, 2 points v/w, coordinates)
# seven pairs of points to define seven lines
V, W = L[:, 0], L[:, 1]
# seven direction vectors of seven lines
R = V - W
R = R / la.norm(R, axis=1, keepdims=True)

# seven projection matrices
RRT = R[:, :, None] @ R[:, None, :]  # or np.einsum('ij,ik->ijk', R, R)
projM = np.eye(6) - RRT
# intersection points of lines with their corresponding orthogonal subspaces
Q = np.einsum('ijk,ik->ij', projM, V)  # or (projM @ V[:, :, None])[..., 0]
print(Q.shape)

# get A, b
A = np.sum(projM, axis=0)
b = np.sum(Q, axis=0)
print(A.shape)
print(b.shape)

In [ ]:
#### solution ####
# QR factorization
# x_QR = ?


#### solution ####

#### solution ####
# psuedo-inverse
# x_SVD = ?


#### solution ####

In [ ]:
# Uncommet and run the following when you are done
# print(np.allclose(x_QR, x_SVD))  # compare solutions from both approaches